In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/files/train.csv")
data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data["Geography"] = le.fit_transform(data["Geography"])
data["Gender"] = le.fit_transform(data["Gender"])

to_drop = ["id","Surname"]
data.drop(to_drop, inplace=True, axis=1)

data = data.dropna(axis = 0)
data

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15674932,668,0,1,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,15749177,627,0,1,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,15694510,678,0,1,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,15741417,581,0,1,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,15766172,716,2,1,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...
165029,15667085,667,2,0,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,15665521,792,0,1,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,15664752,565,0,1,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,15689614,554,2,0,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category = ConvergenceWarning)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipelines = []
pipelines.append(("ScaledLR", Pipeline([("Scaler", StandardScaler()), ("LR", LogisticRegression())])))
pipelines.append(("ScaledLDA", Pipeline([("Scaler", StandardScaler()), ("LDA", LinearDiscriminantAnalysis())])))
pipelines.append(("ScaledKNN", Pipeline([("Scaler", StandardScaler()), ("KNN", KNeighborsClassifier())])))
pipelines.append(("ScaledCART", Pipeline([("Scaler", StandardScaler()), ("CART", DecisionTreeClassifier())])))
pipelines.append(("ScaledNB", Pipeline([("Scaler", StandardScaler()), ("NB", GaussianNB())])))
pipelines.append(("ScaledSVM", Pipeline([("Scaler", StandardScaler()), ("SVM", SVC(max_iter = 300))])))

# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Exited', axis = 1)
Y = data['Exited']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)

# Evaluate each model
results = []
names = []
metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]

for metric in metrics:
   print(metrics[metrics.index(metric)] + ": \n")
   for name, model in pipelines:
       kfold = KFold(n_splits = 10, random_state = 7, shuffle = True)
       cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = metric)
       results.append(cv_results)
       names.append(name)
       msg = (name, cv_results.mean(), cv_results.std( ))
       print(msg)
   print("-" * 80)

Before oversampling:  Counter({0: 104064, 1: 27963})
After oversampling:  Counter({0: 104064, 1: 104064})
--------------------------------------------------------------------------------
accuracy: 

('ScaledLR', 0.7419424357211655, 0.003040064283996543)
('ScaledLDA', 0.7413274306328396, 0.003083676935401919)
('ScaledKNN', 0.8643911281165944, 0.0020130008769542527)
('ScaledCART', 0.8589713909420607, 0.0026228450332330218)
('ScaledNB', 0.7673066416570411, 0.002247141882049544)
('ScaledSVM', 0.5307502324844486, 0.01403756741879787)
--------------------------------------------------------------------------------
precision: 

('ScaledLR', 0.7442681389200552, 0.004665912602146764)
('ScaledLDA', 0.7449251052328096, 0.004668578220235944)
('ScaledKNN', 0.8120296035628767, 0.002647803598308827)
('ScaledCART', 0.8528520802845307, 0.0038637307805049277)
('ScaledNB', 0.765296086283185, 0.0035010863839743133)
('ScaledSVM', 0.6347892035494156, 0.06469821315532061)
------------------------------------

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

In [ ]:
# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Exited', axis = 1)
Y = data['Exited']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train,y_train= SMOTE.fit_resample(X_train,y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)

# Prepare Models
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
models.append(('AB', AdaBoostClassifier()))

metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]

for metric in metrics:
   print(metrics[metrics.index(metric)] + ": \n")
   for name, model in models:
       kfold = KFold(n_splits = 10, random_state = 7, shuffle = True)
       cv_results = cross_val_score(model,X_train,y_train, cv = kfold, scoring = metric)
       msg = (name, cv_results.mean(), cv_results.std( ))
       print(msg)
   print("-" * 80)

Before oversampling:  Counter({0: 104064, 1: 27963})
After oversampling:  Counter({0: 104064, 1: 104064})
--------------------------------------------------------------------------------
accuracy: 

('RF', 0.9085706747229079, 0.0021670676713380246)
('ET', 0.9164696678611408, 0.001902309464914018)
('GBM', 0.8651983240120543, 0.002226843164459934)
('AB', 0.826890174312339, 0.002622243724047725)
--------------------------------------------------------------------------------
precision: 

('RF', 0.9063286134454925, 0.0029826623470822736)
('ET', 0.9029367331657847, 0.003502546416188519)
('GBM', 0.8692775394682414, 0.004030569961368613)
('AB', 0.8303780044539083, 0.004022361631060103)
--------------------------------------------------------------------------------
recall: 

('RF', 0.9124524270277373, 0.0025049567874806016)
('ET', 0.9332894238311166, 0.0019657762394573544)
('GBM', 0.8596702788042476, 0.0014446817334114947)
('AB', 0.8216263957257828, 0.004581044219521377)
---------------------

In [ ]:
## Hyper Parameter Optimization
import xgboost as xgb
# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Exited', axis = 1)
Y = data['Exited']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)


params={
 "max_depth"        : [ 3, 4, 5, 6, 8, 10],
 "min_child_weight" : [ 1, 3],
 "gamma"            : [ 0.0, 0.1, 0.2],
 "colsample_bytree" : [0.7,0.8,0.9],
 "scale_pos_weight" : [1,2,3]
 }
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
model = xgb.XGBClassifier()

random_search=RandomizedSearchCV(model,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

# fitting the RandomizedSearchCV
random_search.fit(X,Y)

# Finding the best estimators
random_search.best_estimator_
# Finding the best param
random_search.best_params_

Before oversampling:  Counter({0: 104064, 1: 27963})
After oversampling:  Counter({0: 104064, 1: 104064})
--------------------------------------------------------------------------------
Fitting 5 folds for each of 5 candidates, totalling 25 fits


{'scale_pos_weight': 2,
 'min_child_weight': 3,
 'max_depth': 3,
 'gamma': 0.1,
 'colsample_bytree': 0.8}

In [ ]:
import xgboost as xgb
# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Exited', axis = 1)
Y = data['Exited']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)


xgb_clf = xgb.XGBClassifier(scale_pos_weight = 2,
                            min_child_weight = 3,
                            max_depth = 3,
                            gamma = 0.1,
                            colsample_bytree = 0.8)
#fitting the model
xgb_clf.fit(X_train,y_train)
# Predicting model
y_predxg = xgb_clf.predict(X_test)

print("The accuracy_score of logit model is:\n", accuracy_score(y_test, y_predxg))
print("-" * 80)
print("The classification report of logit model is:\n", classification_report(y_test, y_predxg))
print("-" * 80)
print("The confusion_matrix of logit model is:\n", confusion_matrix(y_test, y_predxg))
print("-" * 80)
print("The roc_auc_score of logit model is:\n", roc_auc_score(y_test, y_predxg))

Before oversampling:  Counter({0: 104064, 1: 27963})
After oversampling:  Counter({0: 104064, 1: 104064})
--------------------------------------------------------------------------------
The accuracy_score of logit model is:
 0.8100705910867392
--------------------------------------------------------------------------------
The classification report of logit model is:
               precision    recall  f1-score   support

           0       0.93      0.82      0.87     26049
           1       0.53      0.78      0.63      6958

    accuracy                           0.81     33007
   macro avg       0.73      0.80      0.75     33007
weighted avg       0.85      0.81      0.82     33007

--------------------------------------------------------------------------------
The confusion_matrix of logit model is:
 [[21290  4759]
 [ 1510  5448]]
--------------------------------------------------------------------------------
The roc_auc_score of logit model is:
 0.8001447313275903


In [ ]:
#Train other classifiers
et = ExtraTreesClassifier()

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Exited', axis = 1)
Y = data['Exited']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)

grid_space = {'max_depth':[50,75,100], # The maximum depth of the tree
              'n_estimators':[100,150,200], # The number of trees in the forest.
           }

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(et, param_grid = grid_space, cv = 5, scoring = 'accuracy')
model_grid = grid.fit(X_train, y_train)

print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))

Before oversampling:  Counter({0: 104064, 1: 27963})
After oversampling:  Counter({0: 104064, 1: 104064})
--------------------------------------------------------------------------------
Best hyperparameters are: {'max_depth': 75, 'n_estimators': 200}
Best score is: 0.9161046040611313


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Exited', axis=1)
Y = data['Exited']
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)
et= ExtraTreesClassifier(max_depth = 75,
                         n_estimators = 200,
                         n_jobs = 2)
# Fitting the model
et.fit(X_train, y_train)
# Predicting the model
pred_et = et.predict(X_test)

print("The accuracy_score of logit model is:\n", accuracy_score(y_test, pred_et))
print("-" * 80)
print("The classification report of logit model is:\n", classification_report(y_test, pred_et))
print("-" * 80)
print("The confusion_matrix of logit model is:\n", confusion_matrix(y_test, pred_et))
print("-" * 80)
print("The roc_auc_score of logit model is:\n", roc_auc_score(y_test, pred_et))

Before oversampling:  Counter({0: 104064, 1: 27963})
After oversampling:  Counter({0: 104064, 1: 104064})
--------------------------------------------------------------------------------
The accuracy_score of logit model is:
 0.8496682521889296
--------------------------------------------------------------------------------
The classification report of logit model is:
               precision    recall  f1-score   support

           0       0.90      0.91      0.91     26049
           1       0.65      0.62      0.63      6958

    accuracy                           0.85     33007
   macro avg       0.78      0.77      0.77     33007
weighted avg       0.85      0.85      0.85     33007

--------------------------------------------------------------------------------
The confusion_matrix of logit model is:
 [[23729  2320]
 [ 2642  4316]]
--------------------------------------------------------------------------------
The roc_auc_score of logit model is:
 0.7656151339079263
